In [9]:
using KFactors, Pipe, DataFrames, StatsBase, Dates

In [2]:
full_data = read_data("../data/peaks_merged.parquet");

┌ Info: Removing sensors days with peak-hour occ above 99th percentile (22.90%)
└ @ KFactors /Users/mwbc/git/peak-spreading/src/computation.jl:67


## Reasons for inverted U

I'm guessing it has to do with imputation methods. Sensors that never reported valid data are probably imputed differently.

In [4]:
names(full_data)

23-element Vector{String}:
 "station"
 "peak_hour_occ"
 "peak_hour_occ_avg"
 "peak_hour_flow"
 "occ_entropy"
 "occ_entropy_daytime"
 "total_occ"
 "hours_of_congestion"
 "total_flow"
 "station_type"
 "freeway_number"
 "periods_imputed"
 "longest_imputed_time"
 "direction"
 "day_of_week"
 "District"
 "urban"
 "Lanes"
 "Latitude"
 "Longitude"
 "has_missing"
 "date"
 "peak_hour_start"

In [4]:
stdata = @pipe groupby(full_data, :station) |> combine(_,
    :periods_imputed => (x -> sum(288 .- x)) => :periods_observed,
    :date => minimum => :first_seen,
    :date => maximum => :last_seen
);

In [8]:
mean(stdata.periods_observed .== 0)

0.16239754098360656

In [23]:
stdata[stdata.periods_observed .== 0, :station][1:500:end]

4-element Vector{Union{Missing, Int64}}:
  313890
  404281
  774165
 1216187

In [16]:
full_data[full_data.date .== Date(2021, 3, 14), :periods_imputed]

Union{Missing, Int64}[]

In [19]:
unique(Dates.dayname.(unique(full_data.date)))

5-element Vector{String}:
 "Monday"
 "Tuesday"
 "Wednesday"
 "Thursday"
 "Friday"